In [ ]:
import numpy as np

In [ ]:
%matplotlib notebook
from notebookcam_utils import process_on_webcam

In [ ]:
from src.machinelearningsuite.machinelearningsuite import MachineLearningSuite

## Initialize the machine learning suite

In [ ]:
suite = MachineLearningSuite("webcam", "/media/local/data/shape_predictor_68_face_landmarks.dat")
suite.initialize()

You can "comment out" the reset if it annoys you, by changing the type to 'raw'.

In [ ]:
suite.configuration.reset()

In [ ]:
suite.source.release()

## Create the classes and select the facial landmark parts that define them

In [ ]:
suite.create_classes()

In [ ]:
suite.select_parts()

## Collect training data for each class

In [ ]:
def collect_data_for_class(class_index):
    def collect_frame_for_class(frame):
        frame, landmarks = suite.landmark_detector.get_frame_with_landmarks(frame)
        feature_vector = suite.feature_processor.process(landmarks)
        if feature_vector:
            suite.configuration.set_data_values(class_index, feature_vector)
        return frame
    process_on_webcam(process_function=collect_frame_for_class, final_message="Stopped gathering data for class {}".format(class_index), finalize_function=suite.configuration.save_configuration)

In [ ]:
collect_data_for_class(0)

In [ ]:
collect_data_for_class(1)

# Test the predictions

In [ ]:
suite.normalizer.train()
suite.classifier.train()
suite.configuration.save_configuration()

In [ ]:
def predict():
    def predict_on_frame(frame):
        frame, landmarks = suite.landmark_detector.get_frame_with_landmarks(frame)
        feature_vector = suite.feature_processor.process(landmarks)
        if feature_vector:
            feature_vector = np.asarray(feature_vector).reshape(1, -1)
            feature_vector_normalized = suite.normalizer.normalize(feature_vector)
            prediction = suite.classifier.predict(feature_vector_normalized)
            try:
                predicted_class = suite.configuration.classes[int(prediction[0])]
                label = predicted_class
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, label, (100, 400), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
            except IndexError:
                print("This class has no label yet (class index: {})".format(prediction[0]))
        return frame
    process_on_webcam(process_function=predict_on_frame)

In [ ]:
predict()

## Use the classifier to make attachments appear

You can use the classifier that you just trained by passing its path (./config.pkl) to the constructor of a PredictorInterface.

Rename the file to avoid accidentally overwriting it.